# 文本模板

文本模板是一个很常见的需求,像mvc模式的http服务器就需要模板来映射m层和v层,脚手架工具也需要模板用来生成可用代码,像[airflow](http://blog.hszofficial.site/introduce/2017/05/10/%E5%AE%9A%E6%97%B6%E4%BB%BB%E5%8A%A1%E7%AE%A1%E7%90%86/#%E4%BD%BF%E7%94%A8apache-airflow%E9%85%8D%E7%BD%AE%E5%AE%9A%E6%97%B6%E4%BB%BB%E5%8A%A1)也需要模板来传递参数等

python自带模板功能,只是功能不完善,更多的时候我们使用jinja2做模板引起


## 使用str.format作为模板

我们当然可以使用使用字符串的format函数来做模板,这是最轻量的模板了

In [1]:
temp = "a:{a},b:{b}"

In [2]:
temp.format(a=1,b=2)

'a:1,b:2'

## 使用string.Template作为模板

在format之外,python的标准库`string`也提供了基本的模板功能,模板使用美元符号表示占位符

In [3]:
import string

In [4]:
s = string.Template('$who likes $what')
s.substitute(who='tim', what='kung pao')

'tim likes kung pao'

## 使用jinja2做为模板

上面两种模板只能定义简单的模板,如果模板中需要引入抽象,比如有循环,根据变量来判断生成什么样的结果,有继承关系组合关系,就不合适了,[jinja2](http://docs.jinkan.org/docs/jinja2/)是一个成熟的模板引擎,广泛使用在许多领域,也是著名python网络框架[flask](http://flask.pocoo.org/)的默认模板引擎.

jinja2作为一个主要为mvcweb框架服务的模板引擎自然会有很多设置项和针对web框架场景的优化功能,本文不考虑这类问题,单纯将其作为一个模板引擎来介绍

jinja2作为模板引擎,其基本的使用流程是

1. 根据模板语法定义模板字符串
2. 渲染这个字符串

jinja2提供了低级api和高级api:

+ 低级api

In [5]:
from jinja2 import Template
template = Template('Hello {{ name }}!')
template.render(name='John Doe')

'Hello John Doe!'

+ 高级api

In [6]:
from jinja2 import Environment

env = Environment()
template = env.from_string('Hello {{ name }}!')
template.render(name='John Doe')

'Hello John Doe!'

这两种方式都可以正确的渲染模板,高级api更多的是为了管理模板来源.很多时候模板并不是文本而是存在文件中,`Environment`更多的是一个模板文件管理器的角色.本文将以高级api作为主要介绍对象,因为如果要考虑继承,就必须使用高级接口

### 模板语法

模板中的注释使用`{# 这是注释 #}`这样的形式;变量使用`{{ x }}`形式来表示;而语句则使用`{% if x>0 %} {% endif%}`这种形式;

模板语句大致可以分为3类:

+ 流程控制语句

+ 模板复用语句

+ 宏语句

具体的模板语法可以看[文档中的相关描述](http://docs.jinkan.org/docs/jinja2/templates.html#id2),本文只是做基本的介绍

#### 流程控制语句

流程控制语句就是2种

+ 判断分支

基础的二分支语法如下:

```
{% if x>0 %}
...
{% else %}
...
{% endif %}
```

In [7]:
from jinja2 import Environment

env = Environment()
template = env.from_string(
"""{% if x>0 %}x>0{% else %}x<0{% endif %}"""
)
template.render(x=1)

'x>0'

In [8]:
template.render(x=-1)

'x<0'

多分支语法和python类似依然使用if而不是switch

```
{% if x%3 == 0 %}
...
{% elif x%3 == 1 %}
....
{% else %}
...
{% endif %}
```

In [9]:
from jinja2 import Environment

env = Environment()
template = env.from_string(
"""{% if x%3 == 0 %}x%3==0{% elif x%3 == 1 %}x%3==1{% else %}x%3==2{% endif %}"""
)
template.render(x=1)

'x%3==1'

In [10]:
template.render(x=2)

'x%3==2'

In [11]:
template.render(x=6)

'x%3==0'

+ 循环

基础的循环形式如下:

```
{% for i in range(10) %}
{{ i }}
{% endfor %}
```

In [12]:
from jinja2 import Environment

env = Environment()
template = env.from_string(
"""{% for i in range(x) %}{{i}}{% endfor %}"""
)
template.render(x=5)

'01234'

在此之上也可以使用类似python列表解析的语法:

```
{% for i in range(10) if i%2 == 0%}
{{ i }}
{% endfor %}
```

In [13]:
from jinja2 import Environment

env = Environment()
template = env.from_string(
"""{% for i in range(x) if i%2 == 0%}{{ i }}{% endfor %}"""
)
template.render(x=5)

'024'

然后我们也可以递归的遍历


In [14]:
from jinja2 import Environment

env = Environment()
template = env.from_string(
"""{% for i in x recursive %}{%- if i is iterable -%}{{ loop(i) }}{% else %}{{ i }}{%- endif %}{% endfor %}
"""
)
template.render(x=[[1,2,3],4,5,[6,7,8]])

'12345678'

#### 模板复用语句

模板复用依赖于[Loader](http://docs.jinkan.org/docs/jinja2/api.html?highlight=loader#loaders),通常会用到复用自然文本都保存在文件中,这边为了演示清楚我们就用`DictLoader`来代替.

用于模块化的语句有3种:

+ block语句用于标识出可以被继承替换的段落,如果不被替换其中的内容就会被渲染
```
{% block head %}
    ...
{% endblock head%}
```

+ extends语句用于申明继承关系,基模板不需要申明,但自模板必须申明继承自哪个父模板.jinja2是单继承模型


```
{% extends "base.html" %}
...
```

+ super变量,用于获取父模板中对应块的内容

```
{% block head %}
   {{ super() }}
{% endblock head%}
```
+ include语句用于表示包含关系,类似C语言中的include


In [15]:
from jinja2 import DictLoader, Environment

base = """<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN">
<html lang="en">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock title%} - My Webpage</title>
    {% endblock head%}
</head>
<body>
    <div id="content">{% block content %}{% endblock content%}</div>
    <div id="footer">
        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock footer%}
    </div>
</body>"""
child = """{% extends "base.html" %}
{% block title %}Index{% endblock title%}
{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock head%}
{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome on my awesome homepage.
    </p>
{% endblock content%}

{% block footer %}
    {{ super() }}
{% endblock footer%}
"""
loader = DictLoader({
    'base.html': base,
    'child.html':child
})


env = Environment(loader=loader)
print(env.get_template("child.html").render())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN">
<html lang="en">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    
    
    <link rel="stylesheet" href="style.css" />
    <title>Index - My Webpage</title>
    
    <style type="text/css">
        .important { color: #336699; }
    </style>

</head>
<body>
    <div id="content">
    <h1>Index</h1>
    <p class="important">
      Welcome on my awesome homepage.
    </p>
</div>
    <div id="footer">
        
    
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        

    </div>
</body>


In [16]:
print(env.get_template("base.html").render())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN">
<html lang="en">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    
    <link rel="stylesheet" href="style.css" />
    <title> - My Webpage</title>
    
</head>
<body>
    <div id="content"></div>
    <div id="footer">
        
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        
    </div>
</body>


#### 宏语句

jinja2中的宏就类似各种编程语言中的函数,它们用于把常用行为作为可重用的函数,取代手动重复的工作.

宏定义使用`macro`关键字
```
{% macro input(name) %}
input_{{ name }}
{% endmacro %}
```

调用则需要import进宏,之后和变量一样使用`{{}}`
```
{% from 'macro.html' import input %}
{{ input('hsz') }}
```

jinja2的import语法类似python,可以直接import一个文件也可以在文件中导入特定的宏,而且可以使用`as`关键字为其改名

In [17]:
from jinja2 import DictLoader, Environment

macro = """
{% macro input(name) %}
input_{{ name }}
{% endmacro %}
"""

user = """
{% from 'macro.html' import input %}
使用宏input
{{ input(name) }}
"""
loader = DictLoader({
    'macro.html': macro,
    'user.html':user
})


env = Environment(loader=loader)
print(env.get_template("user.html").render(name='hsz'))



使用宏input

input_hsz

